<h1>IFLA WLIC - 2022</h1>
<h2><span style="font-size:11pt;font-family:Arial;color:#000000;background-color:transparent;font-weight:700;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">From text to data inside bibliographic records. Entity recognition and entity linking of contributors and their roles from statements of responsibility</span></h2>

<h2>Imports - Necessary libraries</h2>

In [ ]:
# Formats
import csv
import unidecode
import pandas as pd

# Functions
import re
from fuzzywuzzy import fuzz
from Levenshtein import distance as levenshtein_distance

#Visualisation
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Models
import spacy
from spacy import displacy
from sklearn.neighbors import KNeighborsClassifier

#Data pre-treatment
from texthero import preprocessing
from sklearn.preprocessing import LabelEncoder
import texthero as hero
from sklearn.model_selection import train_test_split

<h2>Functions</h2>

In [ ]:
def name_match(name1,name2):
    titreCivilite=0
    score=0
    name1=name1.replace("-"," ")
    name1 = re.sub(r"(.[.])()(.[.])",r"\1 \3", name1)
    name2=name2.replace("-"," ")
    name_components_1 = name1.split(" ")
    name_components_2 = name2.split(" ")
    nameComponent1Match={}
    nameComponent2Match={}
    for index in range(len(name_components_1)):
        name_component1=unidecode.unidecode(name_components_1[index]).lower()
        ##TITLE##
        if(index==0 and (name_component1 in ["m.","mme.","dr."])):
            titreCivilite=1
            score+=1
        ##ABREVIATIION##
        elif(len(name_component1)<3 or ("." in name_component1)):
            for index2 in range(len(name_components_2)):
                name_component2=unidecode.unidecode(name_components_2[index2]).lower()
                if(name_component2!=''):
                    match=True
                    for i in range(0,len(name_component1)):
                        if(i<len(name_component2)and name_component1[i]!=name_component2[i] and name_component1[i]!="."):
                            match=False
                    if(match):
                        if(name_component1 not in nameComponent1Match.keys()):
                            nameComponent1Match[name_component1]=1
                        else:
                            nameComponent1Match[name_component1]+=1
                        if(name_component2 not in nameComponent2Match.keys()):
                            nameComponent2Match[name_component2]=1
                        else:
                            nameComponent2Match[name_component2]+=1
        ##Complete Name##
        else:
            for index2 in range(len(name_components_2)):
                    name_component2=unidecode.unidecode(name_components_2[index2]).lower()
                    ratio = fuzz.ratio(name_component1,name_component2)
                    if(name_component2!=''):
                        if(name_component1==name_component2 or (ratio>=80)):
                            if(name_component1 not in nameComponent1Match.keys()):
                                alignment=''
                                for key in nameComponent1Match.keys():
                                    if(fuzz.ratio(name_component1,key)>=80):
                                        alignment=key
                                if(alignment==''):
                                    nameComponent1Match[name_component1]=1
                                else:
                                    nameComponent1Match[alignment]+=1
                            else:
                                nameComponent1Match[name_component1]+=1
                            if(name_component2 not in nameComponent2Match.keys()):
                                alignment=''
                                for key in nameComponent2Match.keys():
                                    if(fuzz.ratio(name_component2,key)>=80):
                                        alignment=key
                                if(alignment==''):
                                    nameComponent2Match[name_component2]=1
                                else:
                                    nameComponent2Match[alignment]+=1
                            else:
                                nameComponent2Match[name_component2]+=1                        
    for key in nameComponent1Match.keys():
        score+=nameComponent1Match[key]    
    
    #print("Matching between "+name1+" and "+name2+" has a score of "+str(score))
    if(score==len(name_components_1) and (len(nameComponent2Match.keys())+titreCivilite)==(len(name_components_1))):
        return True
    else:
        return False
    
def create_trainingSet(load_name,save_name):
    model = spacy.load("./Model_NER_PERS_FONCT_10K")
    rows=[]
    file = open("./data/"+load_name,mode='r',encoding="utf8")
    csvreader = csv.reader(file, delimiter=",")
    header = next(csvreader)
    dicoLabelAmt={}
    #############For ever bibliographical record entry in the csv file#############
    for row in csvreader:
        ppn=row[1]
        bibliographicalRecord = row[2]
        accessPoints = row[3]
        documentSupports = row[4]
        ppnLinks = row[5]
        textualRessources = row[6]
        mentions = bibliographicalRecord.split(" / ",1)
        ###############################Creating table of supports#############################
        supports = documentSupports.split(",")
        ######################################################################################
        ##########################Creating access points dictionary##########################
        dictionnaireAccessPoints = {}
        codes=[]
        accessPoint = accessPoints.split(" ")
        name=""
        for token in accessPoint :
            if(token.isnumeric() and len(name)==0):
                codes.append(token)
            elif(token.isnumeric()):
                #Checking the person is associate to only a single UNIMARC code function to avoi ambiguity
                if (len(name)>0 and len(codes)==1):
                    dictionnaireAccessPoints[name]=codes
                codes=[]
                name=""
                codes.append(token)
            elif(len(name)==0):
                name+=token
            else :
                name+=" "
                name+=token
        #Checking the person is associate to only a single UNIMARC code function to avoi ambiguity
        if (len(name)>0 and len(codes)==1):
            dictionnaireAccessPoints[name]=codes
        ########################################################################################
        ###################################Feature 'Thesis'#####################################
        isThesis = False
        for link in ppnLinks.split(','):
            if(link=="027253139"):
                isThesis=True
        for ressource in textualRessources.split(','):
            if(('m' in ressource[4:8])or('v' in ressource[4:8])):
                isThesis=True
        ########################################################################################
        ####################################Extracting LABEL####################################
        if(len(mentions)>=2):
            ###Position of the mention###
            position=0
            ###Iterating through the mention of responsibilities###
            for mention in mentions[1].split(" ; "):
                dictPersonFunct={}
                doc = model(mention)
                ###Pairing function keywords to their appropriate persons###
                persons=[]
                keywords=[]
                currentFunct=''
                switch=0
                for ent in doc.ents:
                    if(currentFunct==""):
                        currentFunct=ent.label_
                    if(currentFunct!=ent.label_):
                        currentFunct=ent.label_
                        switch+=1
                    if(switch%2==0 and switch>0):
                        for person in persons:
                            if(person in dictPersonFunct.keys()):
                                dictPersonFunct[person].append(keywords)
                            else:
                                dictPersonFunct[person]=keywords
                        persons=[]
                        keywords=[]
                    if(ent.label_== "PER"):
                        persons.append(ent)
                    if(ent.label_ == "FONCT"):
                        keywords.append(ent)
                for person in persons:
                    if(person in dictPersonFunct.keys()):
                        dictPersonFunct[person].append(keywords)
                    else:
                        dictPersonFunct[person]=keywords
                ############################################################
                for person in dictPersonFunct.keys():
                    for key in dictionnaireAccessPoints.keys():
                        if(name_match(person.text, key)):
                            ###Counting amount of entries per function###
                            if(dictionnaireAccessPoints[key][0] in dicoLabelAmt.keys()):
                                dicoLabelAmt[dictionnaireAccessPoints[key][0]]+=1
                            else:
                                dicoLabelAmt[dictionnaireAccessPoints[key][0]]=1
                            ############################################
                            rows.append([ppn,person.text,dictPersonFunct[person],position,supports,isThesis,dictionnaireAccessPoints[key][0]])
                ###incrementing position of the mention###
                position+=1
    ########################################################################################
    ##################################Writing training dataset##############################
    file2 = open("./data/TRAINING_PPN_PERSON_Keywords_Position_DocSupport_Thesis"+save_name+".csv",mode='w',encoding="utf8",newline='')
    writer = csv.writer(file2)
    writer.writerow(["PPN","Person","Keywords","Position","Document Support","Thesis","Label"])
    for row in rows:
        writer.writerow(row)
    print("There are "+str(len(rows))+" entries")
    print("Final count of training entries per function :")
    for key in dicoLabelAmt.keys():
        print("Function "+str(key)+" has "+str(dicoLabelAmt[key])+" entries.")
        
def encodage_données(dataframe,featuresBinaires, featuresOrdinaux, featuresNominaux, featuresTextes):
    encodedDataframe = dataframe.copy()
    ###Relocating target##
    labelColumn = encodedDataframe["Label"] 
    del encodedDataframe["Label"]
    binaryEncoders=[]
    ordinalEncoders=[]
    nominalEncoders=[]
    textEncoders=[]
    #####
    clean_pipeline=[preprocessing.lowercase,preprocessing.remove_whitespace,preprocessing.remove_whitespace,preprocessing.remove_square_brackets]
    #####
    ###Categorical Features###
    for binaryFeature in featuresBinaires:
        print("Encoding Binary Feature : "+str(binaryFeature))
        label_encoder = LabelEncoder()
        label_encoder.fit(encodedDataframe[binaryFeature])
        encodedDataframe[binaryFeature]=label_encoder.transform(dataframe[binaryFeature])
        binaryEncoders.append(label_encoder)
    for ordinalFeature in featuresOrdinaux:
        pass
        #IMPLEMENT
    for nominalFeature in featuresNominaux:
        print("Encoding Nominal Feature : "+str(nominalFeature))
        encodedDataframe = pd.get_dummies(encodedDataframe, prefix = [nominalFeature], columns = [nominalFeature])
        nominalEncoders.append(nominalFeature)
    ###Text Features###
    for textFeature in featuresTextes:
        print("Encoding Text Feature : "+str(textFeature))
        #######CLEAN UP#########
        encodedDataframe[textFeature] = [n.replace('[','') for n in encodedDataframe[textFeature]]
        encodedDataframe[textFeature] = [n.replace(']','') for n in encodedDataframe[textFeature]]
        encodedDataframe[textFeature] = hero.clean(encodedDataframe[textFeature],clean_pipeline)
        ########################
        ###Creating TF-IDF df###
        encodedDataframe[textFeature]=hero.tfidf(encodedDataframe[textFeature], max_features=3000)
        allwordsAmt=len(encodedDataframe[textFeature][0])
        lsts=list(encodedDataframe[textFeature])
        column_names=[]
        for i in range(0,allwordsAmt):
            column_names.append(str(textFeature)+"_"+str(i))
        dataframeTmp = pd.DataFrame(lsts, columns =column_names, dtype = float)
        ########################
        del encodedDataframe[textFeature]
        encodedDataframe = pd.concat([encodedDataframe.reset_index(drop=True),dataframeTmp.reset_index(drop=True)], axis=1)
    ###Relocating target###
    #encodedDataframe["Label"]=labelColumn
    #######################
    return encodedDataframe,labelColumn,binaryEncoders, ordinalEncoders, nominalEncoders, textEncoders

def Matrice_Confusion(Y_test,Y_pred,name):
    ### Confusion Matrix
    cm = confusion_matrix(Y_test,Y_pred)
    ## Get Class Labels
    labels=[]
    for string in Y_test:
        if(string not in labels):
            labels.append(string)
    labels.sort()
    print(labels)
    print(len(labels))
    class_names = labels
    # Plot confusion matrix in a beautiful manner
    fig = plt.figure(figsize=(10, 10))
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax, fmt = 'g'); #annot=True to annotate cells
    # labels, title and ticks
    ax.set_xlabel('Predicted', fontsize=20)
    ax.xaxis.set_label_position('bottom')
    plt.xticks(rotation=90)
    ax.xaxis.set_ticklabels(class_names, fontsize = 10)
    ax.xaxis.tick_bottom()

    ax.set_ylabel('True', fontsize=20)
    ax.yaxis.set_ticklabels(class_names, fontsize = 10)
    plt.yticks(rotation=0)

    plt.title('Matrice de Confusion', fontsize=20)
    
    pyplot.savefig("./confusion_"+name+".png")
    
    plt.close('all')
    
def visualisation_erreurs(df,X_test_PPN,Y_pred,Y_test,name):
    Label_compare=Y_test.to_frame()
    Label_compare["Prediction"]=Y_pred
    Label_compare["PPN"]=X_test_PPN["PPN"]
    Label_compare["Personne"]=X_test_PPN["Personne"]
    fichier2 = open("./data/csv/erreurs_"+name+".csv", 'w')
    writer = csv.writer(fichier2)
    writer.writerow(["PPN","Personne","Keywords","Position","Type","These","Predicted_Label","True_Label"])
    for ind in Label_compare.index:
        if(Label_compare['Label'][ind]!=Label_compare['Prediction'][ind]):
            writer.writerow([Label_compare['PPN'][ind],Label_compare['Personne'][ind],df['Position'][ind],df['Keywords'][ind],df['Type'][ind],df['These'][ind],Label_compare['Prediction'][ind],Label_compare['Label'][ind]])

<h2>Demonstration</h2>

<h3>NER (Named Entity Recognition) Model</h3>

In [ ]:
model = spacy.load("./Model_NER_PERS_FONCT_10K")
doc = model("écrit par Zac Thompson et Emily Horn, dessiné par Alberto Albuquerque")
displacy.render(doc, style="ent")

<h3>Creating training sets</h3>

In [ ]:
true_name1=["F. Nietzsche","Alberto Albuquerque","M. Zaragoza","Mme. Zaragoza","Thomâs Zargoza","Mircel Frontignan","J-C Van Damme","Th Maximilien","J.L. Istin"]
true_name2=["Friedrich Nietzsche","Alberto Jiménez Albuquerque","Thomas Zaragoza","Zaragoza Thomas","Thomas Zaragoza","Marcel Frontagnan","Jean-Claude Van Damme","Théophile Maximilien","Jean-Luc Istin"]
false_name1=["G. Nietzsche","Frank Nietzsche","Z. Zaragoza","Marcel Frontignan"]
false_name2=["Friedrich Nietzsche","Friedrich Nietzsche","Thomas Zaragoza","Marco Frontignan"]
print("Matching between these should be accepted")
print(" ")
for x in range(len(true_name1)):
    print(true_name1[x]+" and "+true_name2[x]+" are matching: "+str(name_match(true_name1[x],true_name2[x])))
    Token_Set_Ratio = fuzz.token_set_ratio(true_name1[x],true_name2[x])
    print("Token_Set_Ratio is :"+str(Token_Set_Ratio))
print(" ")
print("========================================")
print(" ")
print("Matching between these should be refused")
print(" ")
for x in range(len(false_name1)):
    print(false_name1[x]+" and "+false_name2[x]+" are matching : "+str(name_match(false_name1[x],false_name2[x])))
    Token_Set_Ratio = fuzz.token_set_ratio(false_name1[x],false_name2[x])

In [ ]:
create_trainingSet("ID_PPN_Mentions_AP_DocSup_PPNLink_TextualRessources_TEST.csv","")

<h2>Loading and encoding training set</h2>

In [ ]:
df = pd.read_csv("./data/TRAINING_Classes_13_Sample_1000.csv")
encodedDataframe,targetDataframe,binaryEncoders, ordinalEncoders, nominalEncoders, textEncoders = encodage_données(df,["Thesis"],[],[],["Document support","Keywords"])
X_train,X_test,Y_train,Y_test = train_test_split(encodedDataframe,targetDataframe,test_size=0.2,random_state=0)
X_train_PPN_Personne = X_train[['PPN','Personne']]
X_train =  X_train.drop(['PPN','Personne'], axis=1)
X_test_PPN_Personne  = X_test[['PPN','Personne']]
X_test = X_test.drop(['PPN','Personne'], axis=1)

In [ ]:
df

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test,Y_pred))
print(metrics.classification_report(Y_test, Y_pred, digits=3))
Matrice_Confusion(Y_test,Y_pred,"KNN_13Classes")
visualisation_erreurs(df,X_test_PPN_Personne,Y_pred,Y_test,"Eq_Sample_1000_13")